<a href="https://colab.research.google.com/github/hiraso/lab_task/blob/master/%E7%AC%AC5%E5%9B%9E%E8%AA%B2%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第5回課題
第5回課題では，以下の項目を満たすように分析してください．

明確な解答は用意していませんが，少なくとも間違っていないと言える推論をするようにしてください．

- データの取得
- データの確認
 - 特徴量の確認
 - ラベルの確認
- 仮説を立てる
- データの分析
- モデルの訓練
- 予測
- 精度の確認
- 仮説の確認

# データの取得
wine data 以外を利用してデータ分析してください．

なるべく，ラベルと属性から推論が立てやすいデータを選択するとやりやすいです．

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston

In [0]:
data = load_boston()

# データの確認
データ分析以前にわかることを確認．特徴量の属性やドメイン知識など
- 特徴量の確認
- ラベルの確認

In [262]:
data.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])

In [263]:
data["feature_names"]

array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'TAX', 'PTRATIO', 'B', 'LSTAT'], dtype='<U7')

# 仮説を立てる
データについて，導きたい仮説と結論を整理してください．

住宅価格は、平均の部屋数と固定資産税に依存していると仮定する。

# データの分析

In [264]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
boston_df = pd.DataFrame(mms.fit_transform(data["data"]), columns=data["feature_names"])
boston_df = pd.concat([boston_df, pd.DataFrame(data["target"], columns=["target"])], axis=1)
boston_df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,0.040544,0.113636,0.391378,0.069170,0.349167,0.521869,0.676364,0.242381,0.371713,0.422208,0.622929,0.898568,0.301409,22.532806
std,0.096679,0.233225,0.251479,0.253994,0.238431,0.134627,0.289896,0.191482,0.378576,0.321636,0.230313,0.230205,0.197049,9.197104
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000
25%,0.000851,0.000000,0.173387,0.000000,0.131687,0.445392,0.433831,0.088259,0.130435,0.175573,0.510638,0.945730,0.144040,17.025000
50%,0.002812,0.000000,0.338343,0.000000,0.314815,0.507281,0.768280,0.188949,0.173913,0.272901,0.686170,0.986232,0.265728,21.200000
75%,0.041258,0.125000,0.646628,0.000000,0.491770,0.586798,0.938980,0.369088,1.000000,0.914122,0.808511,0.998298,0.420116,25.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,50.000000


In [265]:
boston_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.000000,0.18,0.067815,0.0,0.314815,0.577505,0.641607,0.269203,0.000000,0.208015,0.287234,1.000000,0.089680,24.0
1,0.000236,0.00,0.242302,0.0,0.172840,0.547998,0.782698,0.348962,0.043478,0.104962,0.553191,1.000000,0.204470,21.6
2,0.000236,0.00,0.242302,0.0,0.172840,0.694386,0.599382,0.348962,0.043478,0.104962,0.553191,0.989737,0.063466,34.7
3,0.000293,0.00,0.063050,0.0,0.150206,0.658555,0.441813,0.448545,0.086957,0.066794,0.648936,0.994276,0.033389,33.4
4,0.000705,0.00,0.063050,0.0,0.150206,0.687105,0.528321,0.448545,0.086957,0.066794,0.648936,1.000000,0.099338,36.2


In [266]:
df_corr = boston_df.corr()
df_corr

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
CRIM,1.000000,-0.200469,0.406583,-0.055892,0.420972,-0.219247,0.352734,-0.379670,0.625505,0.582764,0.289946,-0.385064,0.455621,-0.388305
ZN,-0.200469,1.000000,-0.533828,-0.042697,-0.516604,0.311991,-0.569537,0.664408,-0.311948,-0.314563,-0.391679,0.175520,-0.412995,0.360445
INDUS,0.406583,-0.533828,1.000000,0.062938,0.763651,-0.391676,0.644779,-0.708027,0.595129,0.720760,0.383248,-0.356977,0.603800,-0.483725
CHAS,-0.055892,-0.042697,0.062938,1.000000,0.091203,0.091251,0.086518,-0.099176,-0.007368,-0.035587,-0.121515,0.048788,-0.053929,0.175260
NOX,0.420972,-0.516604,0.763651,0.091203,1.000000,-0.302188,0.731470,-0.769230,0.611441,0.668023,0.188933,-0.380051,0.590879,-0.427321
RM,-0.219247,0.311991,-0.391676,0.091251,-0.302188,1.000000,-0.240265,0.205246,-0.209847,-0.292048,-0.355501,0.128069,-0.613808,0.695360
AGE,0.352734,-0.569537,0.644779,0.086518,0.731470,-0.240265,1.000000,-0.747881,0.456022,0.506456,0.261515,-0.273534,0.602339,-0.376955
DIS,-0.379670,0.664408,-0.708027,-0.099176,-0.769230,0.205246,-0.747881,1.000000,-0.494588,-0.534432,-0.232471,0.291512,-0.496996,0.249929
RAD,0.625505,-0.311948,0.595129,-0.007368,0.611441,-0.209847,0.456022,-0.494588,1.000000,0.910228,0.464741,-0.444413,0.488676,-0.381626
TAX,0.582764,-0.314563,0.720760,-0.035587,0.668023,-0.292048,0.506456,-0.534432,0.910228,1.000000,0.460853,-0.441808,0.543993,-0.468536


# モデルの訓練
データは分割するようにしてください．交叉検証についてはまだ説明したいので自由にしてください．

また，モデルの訓練およびデータ分割時に乱数の指定がないなど再現性がなさそうなコードは認めないことにします．

モデルは自由ですが，sklearn を想定しています．自作ライブラリも可ですが，自作の場合，該当箇所は適当に読みます．

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(boston_df[data["feature_names"]], boston_df["target"], test_size=0.3, random_state=43)

In [274]:
from sklearn.ensemble import RandomForestRegressor  as RFR
clf = RFR(n_jobs=-1, random_state=43)
clf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=43, verbose=0, warm_start=False)

In [207]:
clf = SVC(random_state=43)
clf.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

In [278]:
predict = clf.predict(X_test)
predict

array([24.903, 15.832, 20.585, 30.533, 18.507, 17.212, 18.166, 23.858,
       20.976, 17.242, 21.672, 26.326, 17.727, 20.267, 18.889,  9.534,
       26.829, 33.251, 13.144, 46.852, 20.269,  8.641, 20.212, 34.267,
        8.504, 27.485, 15.762, 19.709, 20.419, 14.2  , 20.826, 20.24 ,
       15.015, 11.721, 19.704, 19.693, 15.061, 20.165, 26.217, 24.842,
       19.404,  9.819, 22.222, 20.131, 46.084, 19.415, 21.678, 18.594,
       43.533, 14.49 , 27.253, 45.503, 44.005, 21.338, 14.763, 48.751,
       42.283, 23.313, 27.793, 26.348, 20.342, 15.28 , 22.083, 23.263,
       29.756, 14.352, 20.416, 34.436,  9.425, 23.229, 16.01 ,  8.651,
        9.123, 12.168, 15.05 , 23.266, 20.204, 22.89 , 23.418, 21.034,
       12.223, 20.177, 14.937,  9.81 , 18.302, 26.836, 12.109, 34.234,
       12.026, 13.987, 15.51 , 20.965, 20.95 , 22.324, 20.639, 18.145,
       32.259, 32.915, 32.452, 43.628, 15.938, 20.337,  9.284, 27.88 ,
       20.826, 15.214, 14.898, 19.609, 19.838, 34.223, 13.197, 26.912,
      

# 予測

In [284]:
pred = clf.predict(X_test)
pred

array([24.903, 15.832, 20.585, 30.533, 18.507, 17.212, 18.166, 23.858,
       20.976, 17.242, 21.672, 26.326, 17.727, 20.267, 18.889,  9.534,
       26.829, 33.251, 13.144, 46.852, 20.269,  8.641, 20.212, 34.267,
        8.504, 27.485, 15.762, 19.709, 20.419, 14.2  , 20.826, 20.24 ,
       15.015, 11.721, 19.704, 19.693, 15.061, 20.165, 26.217, 24.842,
       19.404,  9.819, 22.222, 20.131, 46.084, 19.415, 21.678, 18.594,
       43.533, 14.49 , 27.253, 45.503, 44.005, 21.338, 14.763, 48.751,
       42.283, 23.313, 27.793, 26.348, 20.342, 15.28 , 22.083, 23.263,
       29.756, 14.352, 20.416, 34.436,  9.425, 23.229, 16.01 ,  8.651,
        9.123, 12.168, 15.05 , 23.266, 20.204, 22.89 , 23.418, 21.034,
       12.223, 20.177, 14.937,  9.81 , 18.302, 26.836, 12.109, 34.234,
       12.026, 13.987, 15.51 , 20.965, 20.95 , 22.324, 20.639, 18.145,
       32.259, 32.915, 32.452, 43.628, 15.938, 20.337,  9.284, 27.88 ,
       20.826, 15.214, 14.898, 19.609, 19.838, 34.223, 13.197, 26.912,
      

# 精度の確認

In [283]:
clf.score(X_test, y_test)

0.881687077936032

# 仮説の確認
最初に立てた仮説を確認してください．

予測傾向など精度ベースでも仮説を確認することはできますが，決定木や可視化を用いる方が楽かもしれません．

In [0]:
相関からみて、関係が深いのは平均の部屋数とRM（何かわかりませんでした）でした。